In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import os
import re
import yaml
import sys
import subprocess
import math
import glob
import time
import random
from statistics import mean

In [ ]:
from AstraWrapper.SettingsFile import SettingsFile
from AstraWrapper.Astra import Astra

inFile = SettingsFile("parallelBeam")
astra = Astra(inFile)


# Junkyard for codes
Apart from the AstraWrapper which has implemented functions to run Astra and change settings and so on, for specific measurements, these are the codes which i used. They might not be entirely working because as AstraWrapper was changing, I did not update them. Nonetheless, they are here for anybody's use in the future. Each measurement has it's own function and is run by the cell below.

In [ ]:
def fillTable(inputFile):
    #this is code for measurement described in 3.1 section in report, used for filling out tables 
    
    Dmin = [0.0,0.0]
    Dmax = [1.0,1.0]
    bounds = [(low, high) for low, high in zip(Dmin, Dmax)]
    method = "Powell" #COBYLA
    tolerance = 1e-5
    fields = ["top hat fields", "Astra fringe fields", "field profiles"]
    
    with open(inputFile, "r") as file:
        input = file.readlines()

    setups = []
    for i,line in enumerate(input):
        line = line.replace("\n","")
        line = line.split(" ")  
        num = [float(line[0]), float(line[1]), float(line[2]), float(line[3]), float(line[4])]
        setups.append(num)

    results = [[], [], []]
    #print(setups)
    for i in range(3): 
        astra.quadType(i)
        for setup in setups:
            
            #parallel
            time1 = time.time()
            res1 = sc.optimize.minimize(function, (0.1,0.1), method=method, tol=tolerance, bounds=bounds,args=(setup[0],setup[3],  setup[4], "parallel") )
            time2 = time.time()
            if not res1.success:
                print(f"Did not obtain result for parallel focusing. The result turned out {res1.x}")
                continue 
            astra.plotRefXY(setup[0], *res1.x, setup[3],setup[4], f"Found solution for {fields[i]}, parallel focusing,\nPz= {setup[4]*1e-6} MeV, time= {math.ceil(10*(time2-time1))/10} s:\nD2 = {math.ceil(res1.x[0]*1e+5)/1000} cm, D3 = {math.ceil(res1.x[1]*1e+5)/1000} cm", f"parallel_Pz={setup[4]*1e-6}")
            results[i].append([setup[4]*1e-6, setup[0]*1e+2, 0 ,res1.x[0]*1e+2, res1.x[1]*1e+2] + findInfo(setup[0], *res1.x, setup[3], setup[4]))
            
            #point
            time1 = time.time()
            res2 = sc.optimize.minimize(function, (0.1,0.1), method=method, tol=tolerance, bounds=bounds,args=(setup[0], setup[3], setup[4], "point"))
            time2 = time.time()
            if not res2.success:
                print(f"Did not obtain result for point focusing. The result turned out {res2.x}")
                continue
            astra.plotRefXY(setup[0], *res2.x, setup[3],setup[4], f"Found solution for {fields[i]}, point-point focusing,\nPz = {setup[4]*1e-6} MeV, time= {math.ceil(10*(time2-time1))/10} s: D2 = {math.ceil(res2.x[0]*1e+5)/1000} cm, D3 = {math.ceil(res2.x[1]*1e+5)/1000} cm")#, f"pointpoint_Pz={setup[4]*1e-6}")
            results[i].append([setup[4]*1e-6, setup[0]*1e+2, setup[3]*1e+2 ,res2.x[0]*1e+2, res2.x[1]*1e+2] + findInfo(setup[0], *res2.x, setup[3], setup[4]))
            
            #x line
            time1 = time.time()
            res3 = sc.optimize.minimize(function, (0.1,0.1), method=method, tol=tolerance, bounds=bounds,args=(setup[0], setup[3], setup[4], "lineX"))
            time2 = time.time()
            if not res3.success:
                print(f"Did not obtain result for line X focusing. The result turned out {res3.x}")
                continue
            astra.plotRefXY(setup[0], *res3.x, setup[3],setup[4], f"Found solution for {fields[i]}, line X focusing,\nPz = {setup[4]*1e-6} MeV, time= {math.ceil(10*(time2-time1))/10} s:\nD2 = {math.ceil(res3.x[0]*1e+5)/1000} cm, D3 = {math.ceil(res3.x[1]*1e+5)/1000} cm")#, f"lineX_Pz={setup[4]*1e-6}")
            results[i].append([setup[4]*1e-6, setup[0]*1e+2, setup[3]*1e+2 ,res3.x[0]*1e+2, res3.x[1]*1e+2] + findInfo(setup[0], *res3.x, setup[3], setup[4]))

            #y line
            time1 = time.time()
            res4 = sc.optimize.minimize(function, (0.1,0.1), method=method, tol=tolerance, bounds=bounds,args=(setup[0], setup[3], setup[4], "lineY"))
            time2 = time.time()
            if not res4.success:
                print(f"Did not obtain result for line Y focusing. The result turned out {res4.x}")
                continue
            astra.plotRefXY(setup[0], *res4.x, setup[3],setup[4], f"Found solution for {fields[i]}, line Y focusing,\nPz = {setup[4]*1e-6} MeV, time= {math.ceil(10*(time2-time1))/10} s: D2 = {math.ceil(res4.x[0]*1e+5)/1000} cm, D3 = {math.ceil(res4.x[1]*1e+5)/1000} cm")#, f"lineY_Pz={setup[4]*1e-6}")
            results[i].append([setup[4]*1e-6, setup[0]*1e+2, setup[3]*1e+2 ,res4.x[0]*1e+2, res4.x[1]*1e+2] + findInfo(setup[0], *res4.x, setup[3], setup[4]))
            print(f"Timing with fields {fields[i]}: the entire process for setup: {setup} took {time2 - time1} s")
        
    df = pd.DataFrame(results[0])
    df.to_csv("topHatFields.csv", index=False)
    
    df = pd.DataFrame(results[1])
    df.to_csv("AstraFringeFields.csv", index=False)

    df = pd.DataFrame(results[2])
    df.to_csv("fieldProfiles.csv", index=False)
    


    return 

In [ ]:

runStudy("inputLists/inputForTable.txt")

In [ ]:
def func(D1, D2,D3, D4, momZ, switcher):
    
    dataCurrent = runRef(D1,D2, D3, D4, momZ, False)
    if dataCurrent == 1:
        return 1E+9
    sumX = parallelFocusing(dataCurrent)
    
    print(D3*1e+2, sumX)

    D3vals[switcher].append(D3*1e+2)
    funcVals[switcher].append(sumX)
    
    return sumX

In [ ]:
def studyOfMaxStep(inputFile):
    # with this junk, i studied the different step size at Astra, function below plots the results
    Dmin = [0.0, 0.0]
    Dmax = [1., 1.]
    bounds = [(low, high) for low, high in zip(Dmin, Dmax)]
    
    
    with open(inputFile, "r") as file:
        input = file.readlines()
    
    
    setups = []
    for line in input:
        line = line.replace("\n","")
        line = line.split(" ")  
        num = [float(line[0]), float(line[1]), float(line[2]), float(line[3]), float(line[4])]
        setups.append(num)
    
    proc = subprocess.Popen(
        ['/bin/bash'], 
        stdin=subprocess.PIPE, 
        stdout=subprocess.PIPE, 
        stderr=subprocess.PIPE, 
        text=True
    )
    
    proc.stdin.write("source /opt/intel/oneapi/setvars.sh\n")
    proc.stdin.flush()
    
    '''
    
    results = [[],[]]
    for setup in setups:
        time1 = time.time()
        #res = sc.optimize.minimize(func, (0.1, 0.1),method="COBYLA", bounds=bounds,tol=1e-4, args=(setup[0],setup[3], setup[4] ))
        print(runRef(*setup, False))
        time2 = time.time()
        #plotRefXY3(setup[0], *res.x, setup[3], setup[4], f"solution: {res.x}")
        print(f"Timing: {time2 - time1}")
        break
        
        
        results[0] = list(D3vals)
        results[1] = list(funcVals)
        D3vals.clear()
        funcVals.clear()
    
        plt.scatter(results[0], results[1], label=methods[0] + " tol=1e-4")
        plt.xlabel("D3 [cm]")
        plt.ylabel("f(D3) [mrad^2]")
        plt.xlim(0,15)
        plt.ylim(0,1)
        plt.legend()
        plt.show()
        plt.scatter(results[0], results[1], label=methods[0] + " tol=1e-4")
        plt.scatter(results[2], results[3], label=methods[1] + " tol=1e-4")
        plt.scatter(results[4], results[5], label=methods[2] + " tol=1e-4")
        plt.xlabel("D3 [cm]")
        plt.ylabel("f(D3) [mrad^2]")
        plt.xlim(7.25,7.4)
        plt.ylim(0,0.00001)
    '''
    timings = []
    Hmax = [0.005, 0.001, 0.0005, 0.0001]
    for h in Hmax:
        astra.changeInputData("H_max",str(h))
        time1 = time.time()
        for i in range(100):
            D3 = (7.25 +i*0.15/100)/100
            if h == 0.005:
                resX= func(setups[0][0], setups[0][1],D3,setups[0][3],setups[0][4], 0)
            elif h == 0.001:
                resX= func(setups[0][0], setups[0][1],D3,setups[0][3],setups[0][4], 1)
            elif h == 0.0005:
                resX= func(setups[0][0], setups[0][1],D3,setups[0][3],setups[0][4],2)
            elif h == 0.0001:
                resX= func(setups[0][0], setups[0][1],D3,setups[0][3],setups[0][4],3)
        
        time2 = time.time()
        timings.append(time2 -time1)
        print(timings)
        '''
        plt.scatter(D3vals, funcVals, label="x angle")
        plt.xlabel("D3 [cm]")
        plt.ylabel("f(D3) [mrad^2]")
        plt.xlim(7.25,7.4)
        plt.ylim(0,0.0002)
        
        plt.legend()
        plt.show()
        
        
        plt.plot(D3vals, funcVals, label="x angle")
        plt.xlabel("D3 [cm]")
        plt.ylabel("f(D3) [mrad^2]")
        plt.xlim(7.25,7.4)
        plt.ylim(0,0.0001)
        
        plt.legend()
        plt.show()
        '''
    #proc.stdin.close()
    #proc.wait()  # This waits for the shell process to terminate

In [ ]:
def plotStepStudy():
    plt.scatter(D3vals[0], funcVals[0], label="H_max = 0.005 ns, t = 6.6 s")
    plt.scatter(D3vals[1], funcVals[1], label="H_max = 0.001 ns, t = 15.0 s")
    plt.scatter(D3vals[2], funcVals[2], label="H_max = 0.0005 ns, t = 25.1 s")
    plt.scatter(D3vals[3], funcVals[3], label="H_max = 0.0001 ns, t = 111.1 s")
    plt.xlabel("D3 [cm]")
    plt.ylabel("f(D3) [mrad^2]")
    plt.xlim(7.25,7.4)
    plt.ylim(0,0.0002)
    
    plt.legend()
    plt.show()
    
    
    plt.plot(D3vals[0][0:-100], funcVals[0][0:-100], label="H_max = 0.005 ns, t = 6.6 s")
    plt.plot(D3vals[1], funcVals[1], label="H_max = 0.001 ns, t = 15.0 s")
    plt.plot(D3vals[2], funcVals[2], label="H_max = 0.0005 ns, t = 25.1 s")
    plt.plot(D3vals[3], funcVals[3], label="H_max = 0.0001 ns, t = 111.1 s")
    plt.xlabel("D3 [cm]")
    plt.ylabel("f(D3) [mrad^2]")
    plt.xlim(7.25,7.4)
    plt.ylim(0,0.0002)
    
    plt.legend()
    plt.show()

In [ ]:
studyOfMaxStep("inputLists/maxStepStudy.txt")
plotStepStudy()

In [ ]:
def minimizationComparison():
    #this function was used to compare the different minimization methods
    '''
    args = sys.argv
    args.pop(0)
    if len(args) != 1:
        print(f"more than 1 argument")
    tolerance = float(args[0])
    '''
    topHatShapedQuads(0)
    #boundaries for D2, D3    
    Dmin = [0.0, 0.0]
    Dmax = [0.4, 0.4]
    bounds = [(low, high) for low, high in zip(Dmin, Dmax)]
    tolerance=1e-2
    D1 = [0.1]
    #for i in range(1,21):
    #    D1.append(i/100)
    
    D2P1, D2P2, D2NM1, D2NM2, D2C1,D2C2 = [], [], [],[], [], []
    D3P1, D3P2, D3NM1, D3NM2, D3C1,D3C2 = [], [], [],[], [], []
    funkValP1, funkValP2, funkValNM1, funkValNM2, funkValC1, funkValC2 = [], [], [],[] ,[],[]
    timeP1, timeP2, timeNM1, timeNM2, timeC1, timeC2 = [], [], [], [], [], []
    
    for d1 in D1:    
        time1 = time.time()
        res = sc.optimize.minimize(func, (0.1, 0.1),method="Powell", bounds=bounds,tol=tolerance, args=( d1, 1, 4.5E+8 ))
        time2 = time.time()
        plotRefXY(d1, *res.x, 1, 4.5E+8)
        print(f"Time to find solution: {time2-time1} s")
        print(f"Number of iterations: {res.niter}")
        #D2P1.append(res.x[0])
        #D3P1.append(res.x[1])
        #funkValP1.append(parallelFocusing(runRef(d1, *res.x, 4.5E+8, False)))
        #timeP1.append(time2 -time1)
        
        time1 = time.time()
        bounds2 = [(res.x[0]-0.05, res.x[0]+0.05),(res.x[1]-0.05, res.x[1]+0.05)]
        res = sc.optimize.minimize(func, (random.uniform(res.x[0]-0.05, res.x[0]+0.05), random.uniform(res.x[1]-0.05, res.x[1]+0.05)),method="Powell", bounds=bounds2,tol=tolerance, args=( d1,1, 4.5E+8 ))
        time2 = time.time()
        plotRefXY(d1, *res.x, 1, 4.5E+8)
        print(f"Time to find solution: {time2-time1} s")
        #D2P2.append(res.x[0])
        #D3P2.append(res.x[1])
        #funkValP2.append(angleCalculation(runRef(d1, *res.x, 4.5E+8, False)))
        #timeP2.append(time2 -time1)
        #------------------------------------------------------------------
        time1 = time.time()
        res = sc.optimize.minimize(func, (0.1, 0.1),method="Nelder-Mead", bounds=bounds,tol=tolerance, args=( d1,1, 4.5E+8 ))
        time2 = time.time()
        plotRefXY(d1, *res.x, 1, 4.5E+8)
        print(f"Time to find solution: {time2-time1} s")
        #D2NM1.append(res.x[0])
        #D3NM1.append(res.x[1])
        #funkValNM1.append(angleCalculation(runRef(d1, *res.x, 4.5E+8, False)))
        #timeNM1.append(time2 -time1)
        
        time1 = time.time()
        bounds2 = [(res.x[0]-0.05, res.x[0]+0.05),(res.x[1]-0.05, res.x[1]+0.05)]
        res = sc.optimize.minimize(func, (random.uniform(res.x[0]-0.05, res.x[0]+0.05), random.uniform(res.x[1]-0.05, res.x[1]+0.05)),method="Nelder-Mead", bounds=bounds2,tol=tolerance, args=( d1,1, 4.5E+8 ))
        time2 = time.time()
        plotRefXY(d1, *res.x, 1, 4.5E+8)
        print(f"Time to find solution: {time2-time1} s")
        #D2NM2.append(res.x[0])
        #D3NM2.append(res.x[1])
        #funkValNM2.append(angleCalculation(runRef(d1, *res.x, 4.5E+8, False)))
        #timeNM2.append(time2 -time1)
        #------------------------------------------------------------------
        
        time1 = time.time()
        res = sc.optimize.minimize(func, (0.1, 0.1),method="COBYLA", bounds=bounds,tol=tolerance, args=( d1, 4.5E+8 ))
        time2 = time.time()
        plotRefXY(d1, *res.x, 1, 4.5E+8)
        print(f"Time to find solution: {time2-time1} s")
        #D2C1.append(res.x[0])
        #D3C1.append(res.x[1])
        #funkValC1.append(angleCalculation(runRef(d1, *res.x, 4.5E+8, False)))
        #timeC1.append(time2 -time1)
        
        time1 = time.time()
        bounds2 = [(res.x[0]-0.05, res.x[0]+0.05),(res.x[1]-0.05, res.x[1]+0.05)]
        res = sc.optimize.minimize(func, (random.uniform(res.x[0]-0.05, res.x[0]+0.05), random.uniform(res.x[1]-0.05, res.x[1]+0.05)),method="COBYLA", bounds=bounds2,tol=tolerance, args=( d1,1, 4.5E+8 ))
        time2 = time.time()
        plotRefXY(d1, *res.x, 1, 4.5E+8)
        print(f"Time to find solution: {time2-time1} s")
        #D2C2.append(res.x[0])
        #D3C2.append(res.x[1])
        #funkValC2.append(angleCalculation(runRef(d1, *res.x, 4.5E+8, False)))
        #timeC2.append(time2 -time1)
    
    results = [D1, 
               D2P1,D3P1, funkValP1, timeP1, 
               D2P2, D3P2, funkValP2, timeP2, 
               D2NM1, D3NM1, funkValNM1, timeNM1, 
               D2NM2, D3NM2, funkValNM2, timeNM2, 
               D2C1, D3C1, funkValC1, timeC1, 
               D2C2, D3C2, funkValC2, timeC2]
    df = pd.DataFrame(results)
    df.to_csv(f"table{tolerance}.csv",index=False)


In [ ]:
def plotsOfMinimizationComparison(file):
    #i think i used this function to plot the results of the prevous function, i am not sure. Have fun

    df = pd.read_csv(file)
    results = df.values.tolist()
    plt.plot(results[0][:], results[3][:], label='Powell', color='blue')
    plt.plot(results[0][:], results[7][:], label='Powell w. constraints', color='red')
    plt.plot(results[0][:], results[11][:], label='Nelder-Mead', color='green')
    plt.plot(results[0][:], results[15][:], label='Nelder-Mead w. constraints', color='yellow')
    plt.plot(results[0][:], results[19][:], label='COBYLA', color='purple')
    plt.plot(results[0][:], results[23][:], label='COBYLA w. constraints', color='pink')

    plt.xlabel('D1 [mm]')
    plt.ylabel('f(D1) [mrad]')
    #plt.xlim(0.085, 0.1)
    #plt.ylim(0., 0.0)
    plt.yscale('log')
    plt.title(f"found minima for tolerance {file.replace('tables/table','').replace('.csv','')}")
    plt.legend()
    plt.show()

    

    plt.plot(results[0][:], results[4][:], label='Powell', color='blue')
    plt.plot(results[0][:], results[8][:], label='Powell w. constraints', color='red')
    plt.plot(results[0][:], results[12][:], label='Nelder-Mead', color='green')
    plt.plot(results[0][:], results[16][:], label='Nelder-Mead w. constraints', color='yellow')
    plt.plot(results[0][:], results[20][:], label='COBYLA', color='purple')
    plt.plot(results[0][:], results[24][:], label='COBYLA w. constraints', color='pink')

    plt.xlabel('D1 [mm]')
    plt.ylabel('time [s]')
    #plt.xlim(0.085, 0.1)
    #plt.ylim(0., 0.0)
    #plt.yscale('log')
    plt.title(f"found minima for tolerance {file.replace('tables/table','').replace('.csv','')}")
    plt.legend()
    plt.show()

    tables = os.listdir("tables/")
    tables = sorted(tables)
    data = {"func values [mrad]": ['Powell', "Powell w. constraints", "Nelder-Mead", "Nelder-Mead w. constraints", "COBYLA", "COBYLA w. constraints" ]}
    df = pd.DataFrame(data)
    data = {"time [s]": ['Powell', "Powell w. constraints", "Nelder-Mead", "Nelder-Mead w. constraints", "COBYLA", "COBYLA w. constraints" ]}    
    df2 = pd.DataFrame(data)
    for table in tables:
        d = pd.read_csv("tables/" + table)
        results = d.values.tolist()            
        
        df['tol = ' + table.replace("table","").replace(".csv","")] = [ mean(results[3][:]), mean(results[7][:]), mean(results[11][:]), mean(results[15][:]), mean(results[19][:]), mean(results[23][:]) ]
        df2['tol = ' + table.replace("table","").replace(".csv","")] = [ mean(results[4][:]), mean(results[8][:]), mean(results[12][:]), mean(results[16][:]), mean(results[20][:]), mean(results[24][:]) ]
        
    return df, df2

In [ ]:
minimizationComparison()
#all the tables with results should be in comparisonTables/ dir
plotsOfMinimizationComparison("table)

In [ ]:
def getEquidistantResults(setupFileName):
    # one of the first functions that ran equidistant intervals for D1, Pz
    #back then, AstraWrapper did not exist and i had all the functions written out
    #to run this, one would need to rewrite it to run the methods of Astra class

    update()

    #boundaries for D2, D3    
    Dmin = [0.0,0.0]
    Dmax = [0.4,0.4]
    bounds = [(low, high) for low, high in zip(Dmin, Dmax)]
    
    D1 = []
    Pz = []
    for i in range(1,4):
        D1.append(0.1*i)
        Pz.append(2E+8 + i*2E+7)

    
    results = ""
    resultsTable = {
        "" : ["D1 [m]", "D2 [m]", "D3 [m]", "Pz [eV]", "F_num [keV]","D [m]"]
    }

    df = pd.DataFrame(resultsTable)
    i = 1

    for d1 in D1:
        print(f"Running {d1}")
        for pz in Pz:
            print(f"Running {pz}")            
            topHatShapedQuads(True)
            res = sc.optimize.minimize(func, (0.15, 0.15),method="Powell", bounds=bounds,tol=1e-8, args=(d1, pz))
            sum = angleCalculation(runRef(d1,*res.x , pz, False))
            results += str(d1) + " " + str(res.x[0]) + " " + str(res.x[1]) + " " + str(pz) + "\n"
            fill4DGraph(d1, res.x[0], res.x[1], pz, sum)
            df['setup ' + str(i)] = [d1, res.x[0], res.x[1], pz, sum, d1+res.x[0] +res.x[1] ]
        
        i += 1
        
    with open(setupFileName,"w") as file:
        file.write(results)

    
    return df
    

In [ ]:
getEquidistantResults("outputName.txt")

In [ ]:
#analytic: 0.10 0.1767908617405159 0.1859304244423013 700000000
def analysisOfSensitivity(D1,D2, D3, momZ, switch):  
    #also old function without AstraWrapper, basically runs and shows when different parameters change, how does the 
    #function value actually change and how sensitive it is
    
    topHatShapedQuads(True)
    update()
    input = [D1, D2, D3, momZ]
    #--------------------------------------------------------------------------------------------------------------------------
    print(f"Varying D1 in range from 1 cm to 10 mikrometers")
    difs = [0.01, 0.005,0.003, 0.002, 0.001, 0.0005,0.0003, 0.0002, 0.0001,0.00005 ,0.00003, 0.00002, 0.00001]
    graphDataMX = []
    graphDataMY = []
    graphDataPX = []
    graphDataPY = []
    if switch == 1:
        for dif in difs:
            dataControl = runRef(D1,D2,D3,momZ, False)
            dataTestP = runRef(D1+dif,D2,D3, momZ, False)
            dataTestM = runRef(D1-dif,D2, D3, momZ, False)
    
            relChangeM = math.fabs(angleCalculation(dataControl)-angleCalculation(dataTestM))*100/angleCalculation(dataControl)
            graphDataMX.append(dif)
            graphDataMY.append(relChangeM)
            relChangeP = math.fabs(angleCalculation(dataControl)-angleCalculation(dataTestP))*100/angleCalculation(dataControl)
            graphDataPX.append(dif)
            graphDataPY.append(relChangeP)        
    
        plt.scatter(graphDataPX, graphDataPY, color='blue', label='D1 + change')
        plt.scatter(graphDataMX, graphDataMY, color='red', label='D1 - change')
        plt.xscale('log')  # Set x-axis to logarithmic scale
        plt.yscale('log')
        plt.xlabel('change [m]')
        plt.ylabel('rel. change in angle sum [%]')
        plt.title(f"varying D1 with input '{input}'")
        plt.legend()
        plt.show()

    elif switch == 2:
        #--------------------------------------------------------------------------------------------------------------------------
        print(f"Varying D2 in range from 1 cm to 10 mikrometers")
        
        for dif in difs:
            dataControl = runRef(D1,D2,D3,momZ, False)
            dataTestP = runRef(D1,D2+dif,D3, momZ, False)
            dataTestM = runRef(D1,D2-dif, D3, momZ, False)
    
            relChangeM = math.fabs(angleCalculation(dataControl)-angleCalculation(dataTestM))*100/angleCalculation(dataControl)
            graphDataMX.append(dif)
            graphDataMY.append(relChangeM)
            relChangeP = math.fabs(angleCalculation(dataControl)-angleCalculation(dataTestP))*100/angleCalculation(dataControl)
            graphDataPX.append(dif)
            graphDataPY.append(relChangeP)        
    
        plt.scatter(graphDataPX, graphDataPY, color='blue', label='D2 + change')
        plt.scatter(graphDataMX, graphDataMY, color='red', label='D2 - change')
        plt.xscale('log')  # Set x-axis to logarithmic scale
        plt.yscale('log')
        plt.xlabel('change [m]')
        plt.ylabel('rel. change in angle sum [%]')
        plt.title(f"varying D2 with input '{input}'")
        plt.legend()
        plt.show()


    elif switch == 3:
        #--------------------------------------------------------------------------------------------------------------------------
        print(f"Varying D3 in range from 1 cm to 10 mikrometers")
        
        for dif in difs:
            dataControl = runRef(D1,D2,D3,momZ, False)
            dataTestP = runRef(D1,D2,D3+dif, momZ, False)
            dataTestM = runRef(D1,D2, D3-dif, momZ, False)
    
            relChangeM = math.fabs(angleCalculation(dataControl)-angleCalculation(dataTestM))*100/angleCalculation(dataControl)
            graphDataMX.append(dif)
            graphDataMY.append(relChangeM)
            relChangeP = math.fabs(angleCalculation(dataControl)-angleCalculation(dataTestP))*100/angleCalculation(dataControl)
            graphDataPX.append(dif)
            graphDataPY.append(relChangeP)        
    
        plt.scatter(graphDataPX, graphDataPY, color='blue', label='D3 + change')
        plt.scatter(graphDataMX, graphDataMY, color='red', label='D3 - change')
        plt.xscale('log')  # Set x-axis to logarithmic scale
        plt.yscale('log')
        plt.xlabel('change [m]')
        plt.ylabel('rel. change in angle sum [%]')
        plt.title(f"varying D3 with input '{input}'")
        plt.legend()
        plt.show()

    elif switch == 4:
        #--------------------------------------------------------------------------------------------------------------------------
        print(f"Varying Pz in range from 50 MeV to 1 eV")
        difMoms = [5e+7, 3e+7 , 1e+7, 5e+6, 3e+6, 1e+6, 5e+5, 3e+5, 1e+5, 5e+4, 3e+4, 1e+4, 5e+3, 3e+3, 1e+3, 5e+2 , 3e+2, 1e+2, 50, 30 ,10, 5, 3, 1] #from 50 MeV to 1 keV
        
        for dif in difMoms:
            dataControl = runRef(D1,D2,D3,momZ, False)
            dataTestP = runRef(D1,D2,D3, momZ+dif, False)
            dataTestM = runRef(D1,D2, D3, momZ-dif, False)
    
            relChangeM = math.fabs(angleCalculation(dataControl)-angleCalculation(dataTestM))*100/angleCalculation(dataControl)
            graphDataMX.append(dif)
            graphDataMY.append(relChangeM)
            relChangeP = math.fabs(angleCalculation(dataControl)-angleCalculation(dataTestP))*100/angleCalculation(dataControl)
            graphDataPX.append(dif)
            graphDataPY.append(relChangeP)        
    
        plt.scatter(graphDataPX, graphDataPY, color='blue', label='Pz + change')
        plt.scatter(graphDataMX, graphDataMY, color='red', label='Pz - change')
        plt.xscale('log')  # Set x-axis to logarithmic scale
        plt.yscale('log')
        plt.xlabel('change [eV]')
        plt.ylabel('rel. change in angle sum [%]')
        plt.title(f"varying Pz with input '{input}'")
        plt.legend()
        plt.show()

    elif switch == 5:
        #--------------------------------------------------------------------------------------------------------------------------
        print(f"Varying initial angle in x and y direction in range from 10 mrad to 0.01 mrad")
        difAngle = [1e-1, 5e-2, 3e-2,1e-2, 5e-3, 3e-3, 1e-3, 5e-4, 3e-4, 1e-4, 5e-5, 3e-5, 1e-5] 
        
        for dif in difAngle:
            print(f"Running '{dif*1000}' mrad")
            changeMom(xmom,ymom,momZ,-1, -1)
            dataControl = runRef(D1,D2,D3,momZ, False)
            changeMom(dif*momZ, dif*momZ, momZ, -1, -1)
            dataTestP = runRef(D1,D2,D3, momZ, False)
    
            if dataControl == 1:
                print("Something is wrong with control data")
                return
            
            if dataTestP == 1:
                relChangeP = 1000
            else:
                relChangeP = math.fabs(angleCalculation(dataControl)-angleCalculation(dataTestP))*100/angleCalculation(dataControl)
                plotRefXY1(D1,D2,D3,momZ,f"dif '{dif*1000}' mrad")
    
            
            graphDataPX.append(dif)
            graphDataPY.append(relChangeP)        
    
        plt.scatter(graphDataPX, graphDataPY, color='blue', label='Vary Px')
        plt.xscale('log')  # Set x-axis to logarithmic scale
        plt.yscale('log')
        plt.xlabel('change [rad]')
        plt.ylabel('rel. change in angle sum [%]')
        plt.title(f"varying Px and Py with input '{input}'")
        plt.legend()
        plt.show()

    return

In [ ]:
def runAnaOffset(D1,D2, D3, momZ):
    #this function varies initial offset, similar to the previous function 
    
    topHatShapedQuads(True)
    update()
    
    input = [D1, D2, D3, momZ]

    difs = [0.005,0.003, 0.001, 0.0005,0.0003, 0.0001,0.00005 ,0.00003, 0.00001, 5.0E-6, 3.0E-6, 1.0E-6,5.0E-7, 3.0E-7, 1.0E-7, 5.0E-8, 3.0E-8, 1.0E-8,5.0E-9, 3.0E-9, 1.0E-9 ]
    #difs = [5.0E-7, 3.0E-7, 1.0E-7, 1E-8, 1E-9, 1E-10 ]

    graphDataMX = []
    graphDataMY = []
    graphDataPX = []
    graphDataPY = []
    
    #--------------------------------------------------------------------------------------------------------------------------
    print(f"Varying initial offset in x and y direction in range from 10 mrad to 0.01 mrad")
    
    for dif in difs:
        print(f"Running '{dif*1000}' mm")
        changeMom(1E+4,1E+4,momZ,0, 0)
        dataControl = runRef(D1,D2,D3,momZ, False)
        changeMom(xmom,ymom, momZ, dif, dif)
        dataTestP = runRef(D1,D2,D3, momZ, False)

        if dataControl == 1:
            print("Something is wrong with control data")
            return
        
        if dataTestP == 1:
            relChangeP = 1000
            relChangeM = 1000
        else:
            relChangeP = math.fabs(angleCalculation(dataControl)-angleCalculation(dataTestP))*100/angleCalculation(dataControl)
            relChangeM = math.fabs(angleCalculation(dataControl)-angleCalculationInverse(dataTestP))*100/angleCalculation(dataControl)
            plotRefXY2(D1,D2,D3,momZ,f"offset '{dif*1000}' mm")

        
        graphDataPX.append(dif*1e+3)
        graphDataPY.append(relChangeP)        
        graphDataMX.append(dif*1e+3)
        graphDataMY.append(relChangeM)   
    
    plt.scatter(graphDataPX, graphDataPY, color='blue', label='x offset in px != 0...')
    plt.scatter(graphDataMX, graphDataMY, color='red', label='y offset in px != 0...')
    plt.xscale('log')  # Set x-axis to logarithmic scale
    plt.yscale('log')
    plt.xlabel('change [mm]')
    plt.ylabel('rel. change in angle sum [%]')
    plt.title(f"varying Px and Py with input '{input}'")
    plt.legend()
    plt.show()



# Beam
From this point on, there are some functions which were used to study beam. I wrote them, though I am not sure if i actually ever ran them and to be ran again, they would have to be rewritten

In [ ]:
def runBeam(D1,D2,D3, momZ, px_sig, py_sig, moreData):


    changePositions(D1,D2,D3)

    #here can modify spreads in x, y, z directions
    updateBeam(-1, -1, px_sig, -1, -1, py_sig, -1, -1 ,momZ )
    
    
    #if moreData, then provide tracking for each of the reference particles and return it for plotting
    outputMoreData = []
    changeInputData("Distribution", fileName + ".ini" )
    
    res = subprocess.run("source /opt/intel/oneapi/setvars.sh > out.txt && ./Astra " + fileName + " > output.txt", shell=True,check=True,executable='/bin/bash' )
    '''
    if res.returncode != 0:
        res = subprocess.run("./Astra " + fileName + " > output.txt", shell=True,check=True,executable='/bin/bash' )
    if res.returncode != 0:
        print(f"Astra returned with an error")
        return 1
    '''
    
    
    dataX = loadDataRef("Xemit")
    dataY = loadDataRef("Yemit")
    
    if moreData:
        return [dataX, dataY]
    else:
        return dataX[-1], dataY[-1]
        

In [ ]:
def divergence(dataX, dataY):

    data = loadDataRef(setupLengthStr)

    p = 0
    for line in data:
        p += (line[3]/line[5])**2 + (line[4]/line[5])**2
        
    
    return math.sqrt(p)    

In [ ]:
def funcBeam(D,D1, mom, sig_px, sig_py):

    data = runBeam( D1, D[0], D[1], mom, sig_px, sig_py, False)
    divSum = divergence(*data)

    return divSum

In [ ]:
def Beam():
# function which each setup runs only once and looks at the outcome of all 
    update()

    #boundaries for D2, D3    
    Dmin = [0.0,0.0]
    Dmax = [0.3,0.3]
    
    bounds = [(low, high) for low, high in zip(Dmin, Dmax)]

    Pz = []
    D1 = []
    for i in range(1,10):
        Pz.append(1.5E+8 + 5E+7*i)
        D1.append(0.01*i)
    
    results = ""
    resultsTable = {
        "" : ["D1 [m]", "D2 [m]", "D3 [m]", "Pz [eV]", "F_num [mrad]", "xAngle_sig [mrad]", "yAngle_sig [mrad]", "active_particles [%]"]
    }

    df = pd.DataFrame(resultsTable)
    i = 1
    for pz in Pz:
        for d1 in D1:
            topHatShapedQuads(True)
            res = sc.optimize.minimize(funcBeam, (0.15, 0.15),method="Powell", bounds=bounds,tol=1e-8, args=(d1, pz, sig_px, sig_py))
            if not res.success:
                results += str(d1) + " " + str(0) + " " + str(0) + " " + str(pz) + "\n"
                continue
            
            sumNum = divergence(*runBeam(d1, *res.x , pz, sig_px, sig_py,False ))
            plotBeam(d1,res.x[0], res.x[1], pz, sig_px, sig_py, f"Numerical solution, [{d1}, {res.x}, {pz}], top hat fields")
            
            results += str(d1) + " " + str(res.x[0]) + " " + str(res.x[1]) + " "  + str(pz) + "\n"
            row = [d1, *res.x, pz, sumNum, sig_px, sig_py, activeParticles()]
            
            df['setup ' + str(i)] = row
            
    i += 1


    with open("resFigs/results.txt","w") as file:
        file.write(results)
    
        
    
    return df
    

In [ ]:
def plotBeam(D1, D2, D3, momZ, px_sig, py_sig, title):
    #this function plots px over py of all particles in a beam

    dataX, dataY = runBeam(D1, D2, D3, momZ,px_sig, py_sig, True)

    x = []
    x_avr = []
    xz = []
    y = []
    y_avr = []
    yz = []
    for line in dataX:
        xz.append(line[0])
        x.append(line[3])
        x_avr.append(line[2])
        
    for line in dataY:
        yz.append(line[0])
        y.append(line[3])   
        y_avr.append(line[2])

    plt.plot(yz, y, label="y rms", color='red')
    #plt.plot(yz, y_avr,label='y avr [mm]', color='yellow')
    plt.plot(xz, x, label="x rms", color='blue')
    #plt.plot(xz, x_avr, label='x avr [mm]', color='green')
    
    plt.xlabel('z [m]')
    plt.ylabel('offset rms [mm]')
    plt.legend()
    plt.title(title)

    plt.show()
    
    return   


In [ ]:
def plotBeam(D1, D2, D3, momZ, px_sig, py_sig, title):
    #this function plots px over py of all particles in a beam

    dataX, dataY = runBeam(D1, D2, D3, momZ,px_sig, py_sig, True)

    x = []
    x_avr = []
    xz = []
    y = []
    y_avr = []
    yz = []
    for line in dataX:
        xz.append(line[0])
        x.append(line[3])
        x_avr.append(line[2])
        
    for line in dataY:
        yz.append(line[0])
        y.append(line[3])   
        y_avr.append(line[2])

    plt.plot(yz, y, label="y rms", color='red')
    #plt.plot(yz, y_avr,label='y avr [mm]', color='yellow')
    plt.plot(xz, x, label="x rms", color='blue')
    #plt.plot(xz, x_avr, label='x avr [mm]', color='green')
    
    plt.xlabel('z [m]')
    plt.ylabel('offset rms [mm]')
    plt.legend()
    plt.title(title)

    plt.show()
    
    return   
